In [ ]:
!pip install openai --quiet
!pip install tiktoken --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.9/224.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0

In [ ]:
from google.colab import drive

drive.mount("/data")

Mounted at /data


In [ ]:
cd /data/MyDrive/ChatGPT/TestPrompts

/data/MyDrive/ChatGPT/TestPrompts


In [ ]:
import numpy as np
import pandas as pd

import openai
import tiktoken

In [ ]:
token = "your_openai_token"  # None
with open("openai.txt", "r") as file:
    token = file.readline().strip()

In [ ]:
data0 = pd.read_csv("explicit_ads.csv")
print(data0.shape)
data0.head()

(59667, 7)


,Text,Id,Length,Nums,Tokens,NumsPerWord,TokensPerWord
0,"продам телевизор mystery, модель mtv-1922lw. ж...","{'platform': 'telegram', 'wall_id': 1525972822...",31,5,90,0.161290,2.903226
1,"продам телевизор supra, модель: stv-lc2222wd, ...","{'platform': 'telegram', 'wall_id': 1525972822...",16,5,55,0.312500,3.437500
2,"продаю новогодний венок, 35 см, полностью ручн...","{'platform': 'telegram', 'wall_id': 1525972822...",23,2,63,0.086957,2.739130
3,"продаю две ёлочки, 30 см, полностью ручная раб...","{'platform': 'telegram', 'wall_id': 1525972822...",16,2,46,0.125000,2.875000
4,продается прилесной участок в коттеджном посел...,"{'platform': 'telegram', 'wall_id': 1525972822...",116,10,431,0.086207,3.715517


In [ ]:
data1 = pd.read_csv("implicit_ads.csv")
print(data1.shape)
data1.head()

(101199, 7)


,Text,Id,Length,Nums,Tokens,NumsPerWord,TokensPerWord
0,деревянная статуэтка 4000 рублей\nигра 400 руб...,"{'platform': 'telegram', 'wall_id': 1525972822...",12,4,43,0.333333,3.583333
1,автокресло britax romer от 6 месяцев до 9 лет ...,"{'platform': 'telegram', 'wall_id': 1525972822...",29,5,97,0.172414,3.344828
2,перчатки christian dior кожаные 8500\n(на одно...,"{'platform': 'telegram', 'wall_id': 1525972822...",13,1,43,0.076923,3.307692
3,"сервиз чайная роза гдр -5500, \nкофейный серви...","{'platform': 'telegram', 'wall_id': 1525972822...",11,2,40,0.181818,3.636364
4,сумка moschino винтаж -20.000 \n\nзабирать на ...,"{'platform': 'telegram', 'wall_id': 1525972822...",7,1,24,0.142857,3.428571


In [ ]:
data = pd.concat([data0, data1])
print("Max tokens per word:", data["TokensPerWord"].max())
data = data[data["TokensPerWord"] < data["TokensPerWord"].quantile(0.999)]
data = data[data["NumsPerWord"] > data["NumsPerWord"].quantile(0.1)]
print(data.shape)
data.set_index(np.arange(len(data)), inplace=True)
data.head()

Max tokens per word: 19.25
(143924, 7)


,Text,Id,Length,Nums,Tokens,NumsPerWord,TokensPerWord
0,"продам телевизор mystery, модель mtv-1922lw. ж...","{'platform': 'telegram', 'wall_id': 1525972822...",31,5,90,0.161290,2.903226
1,"продам телевизор supra, модель: stv-lc2222wd, ...","{'platform': 'telegram', 'wall_id': 1525972822...",16,5,55,0.312500,3.437500
2,"продаю новогодний венок, 35 см, полностью ручн...","{'platform': 'telegram', 'wall_id': 1525972822...",23,2,63,0.086957,2.739130
3,"продаю две ёлочки, 30 см, полностью ручная раб...","{'platform': 'telegram', 'wall_id': 1525972822...",16,2,46,0.125000,2.875000
4,продается прилесной участок в коттеджном посел...,"{'platform': 'telegram', 'wall_id': 1525972822...",116,10,431,0.086207,3.715517


Сразу перемешаем данные

In [ ]:
data = data[np.random.shuffle(np.arange(len(data)))]

Уберём объявления, в которых вместо цены указан диапазон: они нам не интересны, нам нужна конкретная цена

In [ ]:
def detect_price_range(text):
    parts = text.split('\n')
    for part in parts:
        if part.startswith("цена: "):
            return re.match(r'цена: [0-9]+-[0-9]+', part) is not None
    return False

data["Text"].apply(detect_price_range).sum()

18

In [ ]:
print(data.loc[data["Text"].apply(detect_price_range), "Text"].values[0])

описание:  может кому интересны наборы,в лс
контакты: @boni_x
цена: 1450-2050
метро: #рыбацкое
автор: boni_x


In [ ]:
data = data[~data["Text"].apply(detect_price_range)]
data.shape

(1947, 5)

Убрали самых явных "нарушителей, продолжаем

In [ ]:
print("Max tokens per word:", data["TokensPerWord"].max())

Max tokens per word: 6.833333333333333


In [ ]:
data.loc[data["Tokens"] == data["Tokens"].max(), :].head()

,Text,Id,Length,Nums,Tokens,NumsPerWord,TokensPerWord
52584,в брони. срочно продаётся набор туриста full p...,"{'platform': 'telegram', 'wall_id': 1121942433...",306,20,1019,0.065359,3.330065


In [ ]:
N = 10  # Сколько данных хотим разметить

In [ ]:
def count_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    return len(encoding.encode(text))

In [ ]:
client = openai.OpenAI(api_key=token)

In [ ]:
def process_ad(prompt, text):
    return client.chat.completions.create(
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": text},
        ],
        model="gpt-3.5-turbo",
    ).choices[0].message.content

In [ ]:
prompt = """You are an ai trainer in marketplace company. You need to create a JSON from a plain text of a product offer provided by user. This JSON must contain next fields:
"place" - where buyer can recieve product (or delivery info)
"products" - list of dictionaries of valid products.
Each product dict must contain ONLY following information:
"name" - short title for a customer,
"price" - cost of one unit of a product (one number),
"currency" - in what currency is the price indicated (default is 'рубли'),
"size" - size or weight of a product (one string),
"count" - number of product units offered.
Represent different variations on each product as DIFFERENT products.
Do NOT add description and any other fields. Drop all fields with null value.
If NO information about the price provided, the product has to be excluded from the answer.
Answer ONLY with JSON string, no other text is needed.
"""

orig_prompt = """You are an ai trainer in marketplace company. You need to create a JSON from a plain text of a product offer provided by user. This JSON must contain next fields:
"place" - where buyer can recieve product (or delivery info)
"products" - list of dictionaries of valid products.
Each product dict must contain next info:
"name" - short title for a customer,
"price" - cost of a product (one number in rubbles),
"count" - number of product if they are in a pack (weight or lenght if product is uncountable). Drop products without name or price. Answer only with JSON."""

Prompt весит в районе 200 токенов

In [ ]:
data.loc[:10, "Text"].values

array(['продам телевизор mystery, модель mtv-1922lw. жк, 720p hd, диагональ 19" (48 см), hdmi, usb, тип подсветки edge led, есть не значительный дефект в правом нижнем углу в виде темного пятна, 1000₽',
       'продам телевизор supra, модель: stv-lc2222wd, жк, 1080p full hd, диагональ 22" (55 см), hdmi, usb, 3000₽',
       'продаю новогодний венок, 35 см, полностью ручная работа \n2600 р\nесть петля для крепления на стену или дверь\nписать в личные сообщения @paradofcon',
       'продаю две ёлочки, 30 см, полностью ручная работа\nпо 1500 р\nписать в личные сообщения @paradofcon',
       'продается прилесной участок в коттеджном поселке бортневские дачи. 39 км от мкад по ярославскому шоссе. общая площадь 26 сот. + 5 соток с ограничением\nучасток с ровным ландшафтом, сухой, готовый к сделке и к строительству дома,  в отдалении от трассы. \n2 900 000р. от собственника.\n\nв поселке проложены имеются основные коммуникации: электричество(15 квт на домовладение), магистральный газ в 2023г, \

In [ ]:
import re

text = "продаётся коляска в хорошем состоянии, 10 к"
re.sub(r'([0-9]+) ([kKкКтТ]([^a-zA-Zа-яА-ЯёЁ0-9]|$))', r'\1\2', text)

'продаётся коляска в хорошем состоянии, 10к'

In [ ]:
data["Text"] = data["Text"].apply(lambda text: re.sub(r'([0-9]+) ([kKкКтТрР]([^a-zA-Zа-яА-ЯёЁ0-9]|$))', r'\1\2', text))

### **Мой промпт**

In [ ]:
results = []

for text in data.loc[:N, "Text"].values:
  text = re.sub(r'([0-9]+) ([kKкКтТ]([^a-zA-Zа-яА-ЯёЁ0-9]|$))', r'\1\2', text)
  results.append(process_ad(prompt, text))

In [ ]:
for i, res in enumerate(results):
  print(data.loc[i, "Text"])
  print(res)
  print("#" * 50 + '\n\n')

продам телевизор mystery, модель mtv-1922lw. жк, 720p hd, диагональ 19" (48 см), hdmi, usb, тип подсветки edge led, есть не значительный дефект в правом нижнем углу в виде темного пятна, 1000₽
{
  "products": [
    {
      "name": "телевизор mystery",
      "price": 1000,
      "currency": "рубли",
      "size": "19\"",
      "count": 1
    }
  ]
}
##################################################


продам телевизор supra, модель: stv-lc2222wd, жк, 1080p full hd, диагональ 22" (55 см), hdmi, usb, 3000₽
{
  "place": "",
  "products": [
    {
      "name": "телевизор supra, модель: stv-lc2222wd",
      "price": 3000,
      "currency": "рубли",
      "size": "диагональ 22\" (55 см)",
      "count": 1
    }
  ]
}
##################################################


продаю новогодний венок, 35 см, полностью ручная работа 
2600р
есть петля для крепления на стену или дверь
писать в личные сообщения @paradofcon
{
  "place": "личные сообщения @paradofcon",
  "products": [
    {
      "name": "

In [ ]:
import json
from tqdm.notebook import tqdm

In [ ]:
N = 1500

texts, jsons = [], []
for text in tqdm(data.loc[:N, "Text"].values):
    text = re.sub(r'([0-9]+) ([kKкКтТ]([^a-zA-Zа-яА-ЯёЁ0-9]|$))', r'\1\2', text)
    texts.append(text)
    json_response = process_ad(prompt, text)
    jsons.append(json_response)

marked_df = pd.DataFrame({
    "Text": texts,
    "Json": jsons,
})

  0%|          | 0/1501 [00:00<?, ?it/s]

In [ ]:
marked_df.head()

,Text,Json
0,"продам телевизор mystery, модель mtv-1922lw. ж...","{\n ""place"": """",\n ""products"": [\n {\n ..."
1,"продам телевизор supra, модель: stv-lc2222wd, ...","{\n ""place"": ""укажите место получения"",\n ""p..."
2,"продаю новогодний венок, 35 см, полностью ручн...","{\n ""place"": ""писать в личные сообщения @para..."
3,"продаю две ёлочки, 30 см, полностью ручная раб...","{\n ""place"": ""писать в личные сообщения @para..."
4,продается прилесной участок в коттеджном посел...,"{\n ""place"": ""Поселок Бортневские дачи, 39 км..."


In [ ]:
marked_df.to_csv("marked_ads.csv", index=False)